In [ ]:
import tensorflow as tf
import numpy as np

In [ ]:
from shutil import copy
from collections import defaultdict
import scipy
import numpy as np
import matplotlib.pyplot as plt

import random
#import cv2
import glob
from PIL import Image, ImageEnhance
import PIL.ImageOps

#import torch
#import torchvision
#from torch import nn

import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# 0 = all messages are logged (default behavior)
# 1 = INFO messages are not printed
# 2 = INFO and WARNING messages are not printed
# 3 = INFO, WARNING, and ERROR messages are not printed

from tensorflow.keras.datasets import mnist
from tensorflow.keras import layers
import tensorflow_datasets as tfds

# Theano(th)와 Tensorflow(tf) 모두와 호환이 되는 Keras 모듈을 작성
import keras.backend as K
from keras import regularizers
from keras.applications.mobilenet_v2 import MobileNetV2
from keras.models import Model
from keras.layers import Dense, Dropout, Conv2D
from keras.layers import GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator # 데이터 전처리
from keras.callbacks import ModelCheckpoint, CSVLogger
#from keras.optimizers import SGD

from keras_preprocessing.image import array_to_img, img_to_array, load_img

print(tf.test.gpu_device_name())

## 기본 세팅

In [ ]:
!nvidia-smi

## data 불러오기 + split

In [ ]:
import tensorflow_datasets as tfds

(train_ds1,train_ds2, val_ds, test_ds), metadata = tfds.load(
    'tf_flowers',
    split=['train[:40%]','train[40%:80%]', 'train[80%:90%]', 'train[90%:]'],
    with_info=True,
    as_supervised=True,
)

In [ ]:
num_classes = metadata.features['label'].num_classes

print(num_classes)

In [ ]:
metadata.features['label'].names

In [ ]:
metadata.features['label'].str2int

In [ ]:
get_label_name = metadata.features['label'].int2str

print(get_label_name)

In [ ]:
td2=iter(train_ds2)

image, label = next(td2)

print(label)
_ = plt.imshow(image)
_ = plt.title(get_label_name(label))


In [ ]:
IMG_SIZE = 256

resize_and_rescale = tf.keras.Sequential([
  layers.experimental.preprocessing.Resizing(IMG_SIZE, IMG_SIZE),
  layers.experimental.preprocessing.Rescaling(1./255)
])

result = resize_and_rescale(image)
_ = plt.imshow(result)

In [ ]:
data_augmentation = tf.keras.Sequential([
  layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
  layers.experimental.preprocessing.RandomRotation(0.2),
])

In [ ]:
batch_size = 32
AUTOTUNE = tf.data.experimental.AUTOTUNE

def prepare(ds,shuffle=False ,augment=False):
  # Resize and rescale all datasets
    ds = ds.map(lambda x, y: (resize_and_rescale(x), y), 
              num_parallel_calls=AUTOTUNE)

    if shuffle:
        ds = ds.shuffle(1000)

  # Batch all datasets
    ds = ds.batch(batch_size)
    
    
  # Use data augmentation only on the training set
    if augment:
        ds = ds.map(lambda x, y: (data_augmentation(x, training=True), y), 
                num_parallel_calls=AUTOTUNE)

  # Use buffered prefecting on all datasets
    return ds.prefetch(buffer_size=AUTOTUNE)

In [ ]:
train1 = prepare(train_ds1,shuffle=True, augment=True)
train2 = prepare(train_ds2, shuffle=True, augment=True)
validation = prepare(val_ds)
test = prepare(test_ds)

- train1,train2:1468, validation,test: 367

## 모델 기본 세팅

In [ ]:
from deeplab2.model.pixel_encoder import moat

In [ ]:
# releases the global state: avoid clutter from old models and layers
K.clear_session()

## 모델1 Disjoint two model

In [ ]:
from tensorflow.keras.layers import (Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten)
from tensorflow import keras

from AdaBelief_tf import AdaBeliefOptimizer
opt = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

# tf.keras.optimizers.schedules.LearningRateSchedule 나중에. 오목한 형태로 하는게 좋을듯 함.

In [ ]:
nb_train_samples =  1468*2
nb_train1_samples =  1468
nb_train2_samples =  1468  
nb_validation_samples = 367
nb_validation2_samples = 367   
nb_test_samples = 367  
batch_size = 16

### model1

In [ ]:
moat1 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./moat1/model-ckpt-0")
o1 = moat1.output["stage5"]
r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
out1 = tf.keras.layers.Dense(
    units=5, activation='softmax', name='output_layer')(r1)

def create_model():
    return tf.keras.models.Model(moat1.input, out1)

In [ ]:
with tf.device('/device:GPU:0'):
    model1=create_model()
   # model1.load_weights('pet_final_disjoint1_3.hdf5')
  # moat1 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")
    opt2 = AdaBeliefOptimizer(learning_rate=1e-6, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14)
    model1.compile(optimizer=opt2,
                   loss='sparse_categorical_crossentropy', 
                   metrics = tf.keras.metrics.SparseCategoricalAccuracy())
    checkpointer = ModelCheckpoint(filepath='flower_disjoint1_1.hdf5', 
                                   verbose=1, 
                                   monitor ="val_sparse_categorical_accuracy",
                                   save_best_only=True)
    csv_logger = CSVLogger('flower_disjoint1_1.log')
    history = model1.fit_generator(train1,
                    steps_per_epoch = nb_train1_samples // batch_size,      
                    validation_data=validation,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=80,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])


In [ ]:
  moat1.save_weights('pet_moat1_final_disjoint1_4.h5')

In [ ]:
#### 'pet_moat1_final_disjoint1_4.hdf5' 가 91% 정도로 베스트 스코어.

In [ ]:
moat2 = moat.get_model(name='tiny_moat1_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o2 = moat2.output["stage5"]
r2 = tf.keras.layers.Flatten(name='flatten_layer1')(o2)
out2 = tf.keras.layers.Dense(
    units=37, activation='softmax', name='output_layer')(r2)

def create_model2():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  model2 = create_model2()
  model2.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='pet_final_disjoint2_2.hdf5', verbose=1, monitor ="val_sparse_categorical_accuracy",save_best_only=True)
  csv_logger = CSVLogger('pet_final_disjoint2_2.log')
  history = model2.fit_generator(train2_generator,
                    steps_per_epoch = nb_train2_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=80,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])


In [ ]:
  moat2.save_weights('pet_moat2_final_disjoint2_2.h5')

## 앙상블 V1

In [ ]:
moat3 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat4 = moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat3._name="moat3"
moat4._name="moat4"

for layer in moat4.layers:
    layer._name = layer.name + str("_2")


moat3.load_weights('pet_moat1_final_disjoint1.h5')
moat4.load_weights('pet_moat2_final_disjoint2.h5')

moat3.trainable=False
moat4.trainable=False

en_input = Input(shape=(256,256,3))
x = moat3(en_input)
y = moat4(en_input)

b1 = tf.keras.layers.Flatten(name='flatten_layer1')(x["stage5"])
b2 = tf.keras.layers.Flatten(name='flatten_layer2')(y["stage5"])

z= tf.concat([b1, b2],axis=1)

l= Dense(units=512, activation='gelu')(z)
en_out = Dense(units=37, activation='softmax')(l)

def create_model3():
  return tf.keras.models.Model(inputs=en_input, outputs=en_out) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

opt2 = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 0.05,  epsilon=1e-14, clipnorm=1)

with tf.device('/device:GPU:0'):
  en_model = create_model3()
  en_model.compile(optimizer=opt2,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='ensemble_1.hdf5', verbose=1, save_best_only=True)
  csv_logger = CSVLogger('ensemble_1.hdf5.log')
  history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])


In [ ]:
with tf.device('/device:GPU:0'):

 # model1.load_weights('model_fulltrain5.hdf5')
 # model1.layers[1]=moat2

    model1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
    checkpointer = ModelCheckpoint(filepath='model1_disjoint_train1.hdf5', verbose=1, save_best_only=False)
    csv_logger = CSVLogger('model1_disjoint_train1.log')
    history = model1.fit_generator(train1, 
                    steps_per_epoch = nb_train1_samples // batch_size,      
                    validation_data=validation,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=100,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  
    model1.save('model1_disjoint_train1.h5')

In [ ]:
moat2=moat.get_model(name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

o1 = moat2.output["stage5"]

r1 = tf.keras.layers.Flatten(name='flatten_layer1')(o1)
out2 = tf.keras.layers.Dense(
    units=37, activation='softmax', name='output_layer')(r1)

def create_model():
    return tf.keras.models.Model(moat2.input, out2)

model2 = create_model()

In [ ]:
with tf.device('/device:GPU:0'):
    
    model2.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
    checkpointer = ModelCheckpoint(filepath='model_fullset_justflat_2.hdf5', verbose=1, save_best_only=True)
    csv_logger = CSVLogger('model_fullset_justflat_2.log')
    history = model2.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=50,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  
    model2.save('model_fullset_justflat_2.h5')

-------

In [ ]:
model1.trainable=False
model2.trainable=False

In [ ]:
model3=tf.keras.Model(model1.input, model1.layers[53].output)
model4=tf.keras.Model(model2.input, model2.layers[53].output)

In [ ]:
en_input = Input(shape=(256,256,3))
x = model3(en_input)
model5 = tf.keras.Model(en_input, x)

y = model4(en_input)
model6 = tf.keras.Model(en_input, y)

z= tf.concat([model5.output, model6.output],axis=1)
l= Dense(units=4096, activation='relu')(z)
l= Dense(units=2048, activation='relu')(l)
en_outs = Dense(units=37, activation='softmax')(l)

In [ ]:
## 한번만 실행
#for layer in model2.layers:
#    layer._name = layer.name + str("_2")

In [ ]:
opt = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

In [ ]:
def create_model3():
    return tf.keras.models.Model(inputs=en_input, outputs=en_outs) 

#moat2 = moat.get_model(
#    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])
#moat2 = moat._load_moat_pretrained_checkpoint(moat1, path="./model-ckpt-0")

with tf.device('/device:GPU:0'):
    en_model = create_model3()
    en_model.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
    checkpointer = ModelCheckpoint(filepath='ensemble_type1_1.tf', verbose=1, save_best_only=False)
    csv_logger = CSVLogger('ensemble_type1_1.log')
    history = en_model.fit_generator(train_generator, 
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=30,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
    en_model.save('ensemble_type1_1.tf', save_format=tf)

## Model2 Two Disjoint Model

In [ ]:
opt = AdaBeliefOptimizer(learning_rate=1e-5, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

In [ ]:
moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")
out2=moat2.output["res5"]

def create_model():
    return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
    modelv2_1 = create_model()
    modelv2_1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
    checkpointer1 = ModelCheckpoint(filepath='modelv2_disjoint_1.hdf5', verbose=1, save_best_only=False)
    csv_logger1 = CSVLogger('modelv2_disjoint_1.log')
    history = modelv2_1.fit_generator(train1_generator, 
                    steps_per_epoch = nb_train1_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation2_samples // batch_size,  
                    epochs=100,                                             
                    verbose=1,
                    callbacks=[csv_logger1, checkpointer1])
  
    modelv2_1.save('modelv2_disjoint_1.hdf5')

In [ ]:
moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

k11 = moat2.output["stage1"]
k12 = moat2.output["res1"]
k21 = moat2.output["stage2"]
k22 = moat2.output["res2"]
k31 = moat2.output["stage3"]
k32 = moat2.output["res3"]
k41 = moat2.output["stage4"]
k42 = moat2.output["res4"]
k51 = moat2.output["stage5"]
k52 = moat2.output["res5"]

l1 = k11+k12
l2 = k21+k22
l3 = k31+k32
l4 = k41+k42
l5 = k51+k52

f1=Conv2D(32,(3,3),activation='relu')(l1)
f2=MaxPooling2D(2,2)(f1) 
f3=Conv2D(40,(3,3),activation='relu')(f2)
f4=MaxPooling2D(2,2)(f3) 
f5=Conv2D(48,(3,3),activation='relu')(f4)
f6=MaxPooling2D(2,2)(f5) 
f7=Conv2D(56,(3,3),activation='relu')(f6)
f8=MaxPooling2D(2,2)(f7)
f9=Conv2D(64,(3,3),activation='relu')(f8)
f10=MaxPooling2D(2,2)(f9)
f11=Conv2D(72,(2,2),activation='relu')(f10)
ff1=Flatten(name='flatten_layer1')(f11)

g1=Conv2D(32,(3,3),activation='relu')(l2)
g2=MaxPooling2D(2,2)(g1) 
g3=Conv2D(40,(3,3),activation='relu')(g2)
g4=MaxPooling2D(2,2)(g3) 
g5=Conv2D(48,(3,3),activation='relu')(g4)
g6=MaxPooling2D(2,2)(g5) 
g7=Conv2D(56,(3,3),activation='relu')(g6)
g8=MaxPooling2D(2,2)(g7)
g9=Conv2D(64,(2,2),activation='relu')(g8)
gg1=Flatten(name='flatten_layer2')(g9)

h1=Conv2D(64,(3,3),activation='relu')(l3)
h2=MaxPooling2D(2,2)(h1) 
h3=Conv2D(80,(3,3),activation='relu')(h2)
h4=MaxPooling2D(2,2)(h3) 
h5=Conv2D(96,(3,3),activation='relu')(h4)
h6=MaxPooling2D(2,2)(h5) 
h7=Conv2D(112,(2,2),activation='relu')(h6)
hh1=Flatten(name='flatten_layer3')(h7)

i1=Conv2D(128,(3,3),activation='relu')(l4)
i2=MaxPooling2D(2,2)(i1) 
i3=Conv2D(160,(3,3),activation='relu')(i2)
i4=MaxPooling2D(2,2)(i3) 
i5=Conv2D(200,(2,2),activation='relu')(i4)
ii1=Flatten(name='flatten_layer4')(i5)

j1=Conv2D(256,(3,3),activation='relu')(l5)
j2=MaxPooling2D(2,2)(j1) 
j3=Conv2D(320,(3,3),activation='relu')(j2)
jj1=Flatten(name='flatten_layer5')(j3)

fc2=tf.concat([ff1,gg1,hh1,ii1,jj1],axis=1)
out2=Dense(units=101, activation='softmax')(fc2)

def create_model():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  modelv2_1 = create_model()
  modelv2_1.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer1 = ModelCheckpoint(filepath='modelv2_disjoint_2.hdf5', verbose=1, save_best_only=False)
  csv_logger1 = CSVLogger('modelv2_disjoint_2.log')
  history = modelv2_1.fit_generator(train2_generator, 
                    steps_per_epoch = nb_train2_samples // batch_size,      
                    validation_data=validation2_generator,
                    validation_steps=nb_validation2_samples // batch_size,  
                    epochs=200,                                             
                    verbose=1,
                    callbacks=[csv_logger1, checkpointer1])
  
  modelv2_1.save('modelv2_disjoint_2.hdf5')

## 모델 평가 코드

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix,classification_report
batch_size = 32
num_of_test_samples = nb_test_samples

with tf.device('/device:GPU:0'):
  predictions = model1.predict_generator(test_generator)

y_pred = np.argmax(predictions, axis=1)

true_classes = test_generator.classes

class_labels = list(test_generator.class_indices.keys())

print(class_labels)

print(confusion_matrix(test_generator.classes, y_pred))

report = classification_report(true_classes, y_pred, target_names=class_labels)
print(report)

In [ ]:
y_pred==(true_classes+1)

## (Just Flatten)(더미코드)

In [ ]:
tf.debugging.set_log_device_placement(True)

moat2=moat.get_model(
    name='tiny_moat0_pretrain_256_no_pe_1k',input_shape=[256,256,3])

moat2 = moat._load_moat_pretrained_checkpoint(moat2, path="./model-ckpt-0")

from tensorflow.keras.layers import (Input, Dense, concatenate, Conv2D, MaxPooling2D, Flatten)
from tensorflow import keras

from adabelief_tf import AdaBeliefOptimizer
opt = AdaBeliefOptimizer(learning_rate=5e-4, beta_1= 0.9, beta_2=0.999, weight_decay= 1e-4,  epsilon=1e-14, rectify=True)

o11 = moat2.output["stage1"]
o12 = moat2.output["res1"]
o21 = moat2.output["stage2"]
o22 = moat2.output["res2"]
o31 = moat2.output["stage3"]
o32 = moat2.output["res3"]
o41 = moat2.output["stage4"]
o42 = moat2.output["res4"]
o51 = moat2.output["stage5"]
o52 = moat2.output["res5"]

u1 = o11+o12
u2 = o21+o22
u3 = o31+o32
u4 = o41+o42
u5 = o51+o52

r1 = tf.keras.layers.Flatten(name='flatten_layer1')(u1)
r2 = tf.keras.layers.Flatten(name='flatten_layer2')(u2)
r3 = tf.keras.layers.Flatten(name='flatten_layer3')(u3)
r4 = tf.keras.layers.Flatten(name='flatten_layer4')(u4)
r5 = tf.keras.layers.Flatten(name='flatten_layer5')(u5)

v = tf.concat([r1,r2,r3,r4,r5],axis=1)
out2 = tf.keras.layers.Dense(
    units=101, activation='softmax', name='output_layer')(v)

def create_model():
  return tf.keras.models.Model(moat2.input, out2)

with tf.device('/device:GPU:0'):
  model2 = create_model()
  model2.compile(optimizer=opt,
              loss='sparse_categorical_crossentropy', metrics = tf.keras.metrics.SparseCategoricalAccuracy())
  checkpointer = ModelCheckpoint(filepath='best_model_3class_sept.hdf5', verbose=1, save_best_only=True)
  csv_logger = CSVLogger('history.log')
  history = model1.fit_generator(train_generator,
                    steps_per_epoch = nb_train_samples // batch_size,      
                    validation_data=validation_generator,
                    validation_steps=nb_validation_samples // batch_size,  
                    epochs=50,                                             
                    verbose=1,
                    callbacks=[csv_logger, checkpointer])
  
  model2.save('model2_trained.h5')




In [ ]:
from google.colab import files
files.download('model_trained.h5')

In [ ]:
# 초기 변수 설정
n_classes = num_classes

### 이미지 크기 설정
img_width, img_height = 256, 256

train_data_dir = './petdata/train'
train1_data_dir = './petdata/train1'
train2_data_dir = './petdata/train2'
validation_data_dir = './petdata/validation'
validation_data2_dir = './petdata/validation'
test_data_dir = './petdata/test'

nb_train_samples =  1656*2
nb_train1_samples =  1656
nb_train2_samples =  1656   
nb_validation_samples = 368
nb_validation2_samples = 368    
nb_test_samples = 3668  
batch_size = 16

In [ ]:
# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)


train_datagen = ImageDataGenerator(
    rescale=1./ 255,       # multiply the data by the value
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')

train1_datagen = ImageDataGenerator(
    rescale=1./ 255,       # multiply the data by the value
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   

# ImageDataGenerator 객체 생성 (이미지 파일들을 Numpy Array 형태로 가져온 후 증강 기법 적용 준비)
train2_datagen = ImageDataGenerator(
    rescale=1./ 255,       # multiply the data by the value
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,        # Shear angle in counter-clockwise direction as radians
    zoom_range=0.2,         # Range for random zoom. If a float
    horizontal_flip=True,   # Randomly flip inputs horizontally
    fill_mode='nearest')   

val_datagen2 = ImageDataGenerator(rescale=1./ 255)
val_datagen = ImageDataGenerator(rescale=1./ 255)
test_datagen = ImageDataGenerator(rescale=1./ 255)

In [ ]:
# flow_from_directory : Numpy Array Iterator 객체 생성
# 인자로 설정해주는 directory의 바로 하위 디렉토리 이름을 레이블이라고 간주, 그 디렉토리 아래의 파일들을 해당 레이블의 이미지들이라고 알아서 추측


train_generator = train_datagen.flow_from_directory(
    train_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


train1_generator = train1_datagen.flow_from_directory(
    train1_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

train2_generator = train2_datagen.flow_from_directory(
    train2_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')
    
  
validation2_generator = val_datagen.flow_from_directory(
    validation_data2_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')

validation_generator = val_datagen.flow_from_directory(
    validation_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    class_mode='sparse')


test_generator = test_datagen.flow_from_directory(
    test_data_dir,
    target_size=(img_height, img_width),
    batch_size=batch_size,
    shuffle=False,
    class_mode=None)

In [ ]:
#!pip install adabelief-tf